In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://blog.langchain.dev/langgraph/")
docs=loader.load()

In [4]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [5]:
# split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter()
documentes=text_splitter.split_documents(docs)

In [6]:
#indexing
from langchain_community.vectorstores import FAISS
vector=FAISS.from_documents(documentes,embeddings)

In [7]:
retriever=vector.as_retriever()

In [8]:
retriever.get_relevant_documents(
    "what is LangGraph"
)

[Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we\'ve implemented already. We will then highlight a few of the common modifications to these runtimes we\'ve heard requests for, and examples of implementing those. We will finish with a preview of what we will be releasing next.MotivationOne of the big v

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langgraph_search","Search for information about LangGraph. For any question about LangGraph, you must use this tool!")

## creating search Tool

In [14]:
from langchain_community.tools.tavily_search import TavilySearchResults
search=TavilySearchResults()

In [15]:
search.invoke("What is the wetter in Nürnberg")

[{'url': 'https://kachelmannwetter.com/de/wetter/2861650-nuernberg',
  'content': 'Das Wetter in Nürnberg (Bayern, Deutschland): detaillierte Wettervorhersage, 14-Tage-Trend, aktuelles Regenradar bzw. Schneeradar, Niederschlagsprognosen ...'},
 {'url': 'https://www.wetter.com/deutschland/nuernberg/DE0007131.html',
  'content': 'Wie wird das Wetter heute in Nürnberg? Temperatur-, Wind- und Regenvorhersage, sowie aktuelle Wetterwarnungen finden Sie auf wetter.com für 90461 Nürnberg, Bayern, Deutschland.'},
 {'url': 'https://www.accuweather.com/en/de/nuremberg/90402/weather-forecast/167559',
  'content': 'Nuremberg, Bavaria, Germany Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.'},
 {'url': 'https://www.wetteronline.de/wetter/nuernberg',
  'content': 'Nürnberg 21°C. Passau 23°C. Regensburg 22°C. Weiden 22°C. Würzburg 22°C. Zugspitze 6°C. Das Wetter in Nürnberg - Wettervorhersage für heute, morgen und die kommenden Tage mit Wetterberic

In [21]:
tools=[retriever_tool,search]

## creating the Agent

In [16]:
from langchain_openai import ChatOpenAI
model=ChatOpenAI(temperature=0)

In [18]:
# prompt for agent
from langchain import hub
prompt= hub.pull("hwchase17/openai-functions-agent")

In [19]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [22]:
# initializing the agent
from langchain.agents import create_openai_functions_agent
agent=create_openai_functions_agent(model,tools,prompt)

In [23]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(
    agent=agent,tools=tools,verbose=True
)

In [24]:
agent_executor.invoke(
    {
        "input":"Hi!"
    }
)



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'Hi!', 'output': 'Hello! How can I assist you today?'}

In [25]:
agent_executor.invoke(
    {
        "input":"What is the Wetter in Nürnberg"
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Wetter in Nürnberg'}`


[{'url': 'https://kachelmannwetter.com/de/wetter/2861650-nuernberg', 'content': 'Das Wetter in Nürnberg (Bayern, Deutschland): detaillierte Wettervorhersage, 14-Tage-Trend, aktuelles Regenradar bzw. Schneeradar, Niederschlagsprognosen ...'}, {'url': 'https://www.wetteronline.de/wettertrend/nuernberg', 'content': 'Nürnberg 25°C. Passau 26°C. Regensburg 25°C. Weiden 25°C. Würzburg 24°C. Zugspitze 6°C. Das 14-Tage-Wetter für die Region Nürnberg von wetteronline.de.'}, {'url': 'https://www.wetteronline.de/wetter/nuernberg', 'content': 'München 20°C. Nürnberg 21°C. Passau 23°C. Regensburg 22°C. Weiden 22°C. Würzburg 22°C. Zugspitze 6°C. Das Wetter in Nürnberg - Wettervorhersage für heute, morgen und die kommenden Tage mit Wetterbericht und Regenradar von wetteronline.de.'}, {'url': 'https://www.accuweather.com/en/de/nuremberg/90402/weather-forecast/167559', 'content': 'Nu

{'input': 'What is the Wetter in Nürnberg',
 'output': 'I found some sources where you can check the weather in Nürnberg:\n\n1. [Kachelmannwetter](https://kachelmannwetter.com/de/wetter/2861650-nuernberg): Detailed weather forecast, 14-day trend, and current radar for Nürnberg, Bayern, Deutschland.\n2. [Wetteronline.de](https://www.wetteronline.de/wettertrend/nuernberg): 14-day weather forecast for the region of Nürnberg.\n3. [Wetteronline.de](https://www.wetteronline.de/wetter/nuernberg): Weather forecast for today, tomorrow, and the upcoming days in Nürnberg.\n4. [AccuWeather](https://www.accuweather.com/en/de/nuremberg/90402/weather-forecast/167559): Weather forecast for Nuremberg, Bavaria, Germany with current conditions and expectations for the next 3 days.\n5. [Wetter.com](https://www.wetter.com/deutschland/nuernberg/DE0007131.html): Weather forecast for Nürnberg, including temperature, wind, and rain predictions.\n\nYou can visit these websites to get the most up-to-date weather

In [26]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history=[
    HumanMessage(
        content="Can i use LangGraph for agent runtimes?"
    ),
    AIMessage(content="Yes!")
]

In [27]:
agent_executor.invoke(
    {
        "chat_history":chat_history,
        "input":"tell me how"
    }
)



> Entering new AgentExecutor chain...

Invoking: `langgraph_search` with `{'query': 'How to use LangGraph for agent runtimes'}`


TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we've implemented already. We will then highlight a few of the common modifications to these runtimes we've heard requests for, and examples of impl

{'chat_history': [HumanMessage(content='Can i use LangGraph for agent runtimes?'),
  AIMessage(content='Yes!')],
 'input': 'tell me how',
 'output': 'LangGraph is a module built on top of LangChain to better enable the creation of cyclical graphs, which are often needed for agent runtimes. Here is a brief overview of how you can use LangGraph for agent runtimes:\n\n1. **Motivation**: LangGraph allows for the creation of state machines by specifying them as graphs. This is useful for introducing cycles into chains and creating more controlled flows in agent runtimes.\n\n2. **Functionality**:\n   - **StateGraph**: Represents the graph and initializes with a state definition that is updated over time by nodes in the graph.\n   - **Nodes**: Add nodes to the graph using the `graph.add_node(name, value)` syntax, where the value is a function or LCEL runnable that updates the state object.\n   - **Edges**: Connect nodes with edges to create the graph structure, including starting edges, norma